In [2]:
import requests
import pandas as pd
import time
import json

In [3]:
headers={
    'accept': 'application/json',
    'x-api-key':'39d690c5c73a4d1984b3dd0185e07e62'
}

In [1]:
# def get_collections():
#     url = f'https://api.opensea.io/api/v2/collections'
#     params = {
#         'order_by': 'seven_day_volume',
#         'limit': 50,
#     }
#     next_page = False
#     data = []
#     loop =0
#     while loop<=1:
#         try:
#             if next_page:
#                 params['next'] = next_page
#             response = requests.get(url, params=params, headers=headers)
#             response.raise_for_status()
#             collections = response.json().get('collections', [])
#             if collections:
#                 for collection in collections:
#                     data.append({
#                         'collection': collection.get('collection', ''),
#                         'name': collection.get('name', ''),
#                         'owner': collection.get('owner', ''),
#                         'contracts_address': collection['contracts'][0].get('address', '') if 'contracts' in collection and collection['contracts'] else '',
#                         'chain': 'ethereum'
#                     })
#             next_page = response.json().get('next', None)
#             if not next_page:
#                 break
#         except requests.exceptions.RequestException as e:
#             print('Error:', e)
#             break
#         loop =loop+1

#     return data

def get_all_events_by_collection(collection, event_type='sale'):
    base_url = f'https://api.opensea.io/api/v2/events/collection/{collection}'
    params = {'event_type': event_type,
              'after':1717200000,
              'before':1719705600}
    events_data = []
    next_page = False
    loop=0
    while True:
        try:
            if next_page:
                params['next'] = next_page
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            json_data = response.json()
            events = json_data.get('asset_events', [])
            
            if not events:
                print(f'No {event_type} events found for collection {collection}')
                break
            for event in events:
                event_details = {
                    'collection': collection,
                    'event_type': event.get('event_type'),
                    'chain': event.get('chain'),
                    'tokenId': event.get('nft', {}).get('identifier') if isinstance(event.get('nft'), dict) else None,
                    'seller': event.get('seller'),
                    'buyer': event.get('buyer'),
                    'quantity': event.get('quantity'),
                    'price': event.get('payment', {}).get('quantity') if isinstance(event.get('payment'), dict) else None,
                    'decimals': event.get('payment', {}).get('decimals') if isinstance(event.get('payment'), dict) else None,
                    'symbol': event.get('payment', {}).get('symbol') if isinstance(event.get('payment'), dict) else None,
                    'transaction': event.get('transaction'),
                    'event_timestamp': event.get('event_timestamp')
                }
                events_data.append(event_details)
            next_page = json_data.get('next')
            if not next_page:
                break
            
        except requests.RequestException as e:
            print(f'Error fetching data: {e}')
            time.sleep(5)
            continue
        loop = loop+1
        print(loop)

    return events_data

In [ ]:
# collections_data = get_collections()
# collections_data = pd.DataFrame(collections_data)

In [ ]:
# collections_data.to_json('collections.json', orient ='records', indent =3)

In [4]:
# lấy event
data = get_all_events_by_collection('pudgypenguins')
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [5]:
data = pd.DataFrame(data)
data.to_json('data_collection.json',orient = 'records', indent =3)